## 事件流处理器

In [3]:
from illufly.io import EventBlock
EventBlock("Chunk", "hi")

EventBlock(block_type=<chunk>, content=<hi>)

### 实现流交换

基于大模型的AI应用中经常要求模型流式返回，我们有一种实现流输出的标准实现。

In [1]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield "hi\n"
        yield "illufly!\n"

r = MyRun()
r()

hi
illufly!


In [5]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield ["hi\n", "illufly!\n"]

r = MyRun()
r()

['hi\n', 'illufly!\n']

### Runnable 事件流

In [2]:
from illufly.types import Runnable, EventBlock

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield EventBlock("chunk", "hi, ")
        yield EventBlock("chunk", "illufly")
        yield EventBlock("chunk", "!")

def handle(block, **kwargs):
    print(block.json)

r = MyRun(handlers=[handle])
r("hi")

{"id": "9a388e6e9ff911efa4df367dda7bcd4a", "content_id": "9a388ed29ff911efa4df367dda7bcd4a", "block_type": "runnable", "content": "MyRun.4609157120", "created_at": "2024-11-11T14:53:05.484572", "calling_info": {}, "runnable_info": {"name": "MyRun.4609157120", "class_name": "MyRun", "calling_id": "9a388cd4-9ff9-11ef-a4df-367dda7bcd4a"}}
{"id": "9a389eae9ff911efa4df367dda7bcd4a", "content_id": "9a389f809ff911efa4df367dda7bcd4a", "block_type": "chunk", "content": "hi, ", "created_at": "2024-11-11T14:53:05.485000", "calling_info": {}, "runnable_info": {"name": "MyRun.4609157120", "class_name": "MyRun", "calling_id": "9a388cd4-9ff9-11ef-a4df-367dda7bcd4a"}}
{"id": "9a38a2789ff911efa4df367dda7bcd4a", "content_id": "9a38a2c89ff911efa4df367dda7bcd4a", "block_type": "chunk", "content": "illufly", "created_at": "2024-11-11T14:53:05.485081", "calling_info": {}, "runnable_info": {"name": "MyRun.4609157120", "class_name": "MyRun", "calling_id": "9a388cd4-9ff9-11ef-a4df-367dda7bcd4a"}}
{"id": "9a38a

### ChatAgent 事件流

In [5]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM
from illufly.io import log, alog

def handle(block, **kwargs):
    print("[", block.block_type, "]", block.text)

r = FakeLLM(handlers=[handle], sleep=0.5)
r("hi")

[ runnable ] FakeLLM.4609157552
[ user ] hi
[ agent ] FakeLLM.4609157552
[ info ] 记住 10 轮对话
[ info ] I am FakeLLM
[ chunk ] R
[ chunk ] e
[ chunk ] p
[ chunk ] l
[ chunk ] y
[ chunk ]  
[ chunk ] >
[ chunk ] >
[ chunk ]  
[ chunk ] h
[ chunk ] i
[ new_line ] 
[ final_text ] Reply >> hi


'Reply >> hi'

In [9]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM
from illufly.io import log, alog

r = FakeLLM(handlers=[alog], sleep=0.1)
await r("hi")

[AGENT] FakeLLM.4964741232
Reply >> hi


'Reply >> hi'

### 推送事件流

In [12]:
from illufly.chat import FakeLLM

llm = FakeLLM(name="if1")

async for b in llm("给我写一篇200字小作文", generator="async", sleep=0.1):
    print(b)

event: runnable
data: if1

event: user
data: 给我写一篇200字小作文

[AGENT] if1
event: agent
data: if1

event: info
data: 记住 10 轮对话

event: info
data: I am FakeLLM

Revent: chunk
data: R

eevent: chunk
data: e

pevent: chunk
data: p

levent: chunk
data: l

yevent: chunk
data: y

 event: chunk
data:  

>event: chunk
data: >

>event: chunk
data: >

 event: chunk
data:  

给event: chunk
data: 给

我event: chunk
data: 我

写event: chunk
data: 写

一event: chunk
data: 一

篇event: chunk
data: 篇

2event: chunk
data: 2

0event: chunk
data: 0

0event: chunk
data: 0

字event: chunk
data: 字

小event: chunk
data: 小

作event: chunk
data: 作

文event: chunk
data: 文


event: new_line
data: 

event: final_text
data: Reply >> 给我写一篇200字小作文



## 嵌套 BaseAgent

In [1]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen
from illufly.flow import ReAct

def get_city(location: str):
    """由任意地名或地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

store = {}
flow = ReAct(
    ChatQwen(name="react-planner", tools=[get_city, get_weather, booking])
)

for b in flow("我在鸿蒙公司，帮我查询一下天气情况", verbose=False, handlers=[], generator="sync"):
    print(b)

event: runnable
data: ReAct.4765287856

event: node
data: >>> flow-node-1: planner

event: user
data: 我在鸿蒙公司，帮我查询一下天气情况

event: agent
data: react-planner

event: info
data: 记住 10 轮对话

event: chunk
data: **

event: chunk
data: 思考

event: chunk
data: **

event: chunk
data:  


event: chunk
data: 任务要求查询鸿

event: chunk
data: 蒙公司所在地的

event: chunk
data: 天气情况。首先

event: chunk
data: ，我需要确定

event: chunk
data: 鸿蒙公司位于

event: chunk
data: 哪个城市。根据

event: chunk
data: 现有的工具列表，

event: chunk
data: 我可以使用 `get

event: chunk
data: _city` 函数

event: chunk
data: 通过提供鸿蒙

event: chunk
data: 公司的名称或地址

event: chunk
data: 来获取其所在

event: chunk
data: 城市。一旦获得了

event: chunk
data: 城市信息，我

event: chunk
data: 就可以使用 `get

event: chunk
data: _weather` 函数

event: chunk
data: 来查询该城市的

event: chunk
data: 天气情况。

任务

event: chunk
data: 中涉及的关键概念

event: chunk
data: 包括：
- 

event: chunk
data: 鸿蒙公司

event: chunk
data: 位置
  -

event: chunk
data:  城市

event: chunk
data: 

**行动**

event: chunk
data:  
Step1:

event: chunk
data